# Data Preparation and Customer Analytics
# Task 1

Conduct analysis on client's transaction dataset and identify customer purchasing behaviours to generate insights and provide commercial recommendations.

## Work FLow

**1. Examine transaction data;** 

    - Look for inconsistencies
    - missing data across the dataset
    - outliers
    - correctly identify category items
    - numeric data across all tables.
    
    If anomalies are determined, make necessary changes and save it
    
**2. Examine customer data;**

    - Look for inconsistencies
    - missing data across the dataset
    - outliers
    - correctly identify category items
    - numeric data across all tables.

After completing the above tasks, merge the two datasets so it is ready for analysis.

**3. Data analysis and customer segments;**

    - define metrics
    - look at total sales
    - drivers of sales
    - where the highest sales are coming from
    - create charts and graphs
    - note interesting trends and insights
    
**4. Deep dive into customer segments;**

    - define recommendation from insights
    - determine which segments should be targeted
    - form overall conclusion based on analysis
    
**4. Save analysis**

    - save analysis and visuals

We will start this analysis by loading required libraries for analysis

In [1]:
# load libriaries
import numpy as np
import pandas as pd
import seaborn as sns 
import matplotlib.pyplot as plt
%matplotlib inline

In [9]:
pd.read_csv("../data/QVI_purchase_behaviour.csv")

,LYLTY_CARD_NBR,LIFESTAGE,PREMIUM_CUSTOMER
0,1000,YOUNG SINGLES/COUPLES,Premium
1,1002,YOUNG SINGLES/COUPLES,Mainstream
2,1003,YOUNG FAMILIES,Budget
3,1004,OLDER SINGLES/COUPLES,Mainstream
4,1005,MIDAGE SINGLES/COUPLES,Mainstream
...,...,...,...
72632,2370651,MIDAGE SINGLES/COUPLES,Mainstream
72633,2370701,YOUNG FAMILIES,Mainstream
72634,2370751,YOUNG FAMILIES,Premium
72635,2370961,OLDER FAMILIES,Budget


In [10]:
# import data
transaction_df = pd.read_csv("../data/QVI_transaction_data.csv")
customer_df = pd.read_csv("../data/QVI_purchase_behaviour.csv", encoding='utf-8')

UnicodeDecodeError: 'utf-8' codec can't decode bytes in position 0-1: invalid continuation byte

In [13]:
pd.read_csv("../data/QVI_transaction_data.csv")

,DATE,STORE_NBR,LYLTY_CARD_NBR,TXN_ID,PROD_NBR,PROD_NAME,PROD_QTY,TOT_SALES
0,43390,1,1000,1,5,Natural Chip Compny SeaSalt175g,2,6.0
1,43599,1,1307,348,66,CCs Nacho Cheese 175g,3,6.3
2,43605,1,1343,383,61,Smiths Crinkle Cut Chips Chicken 170g,2,2.9
3,43329,2,2373,974,69,Smiths Chip Thinly S/Cream&Onion 175g,5,15.0
4,43330,2,2426,1038,108,Kettle Tortilla ChpsHny&Jlpno Chili 150g,3,13.8
...,...,...,...,...,...,...,...,...
264831,43533,272,272319,270088,89,Kettle Sweet Chilli And Sour Cream 175g,2,10.8
264832,43325,272,272358,270154,74,Tostitos Splash Of Lime 175g,1,4.4
264833,43410,272,272379,270187,51,Doritos Mexicana 170g,2,8.8
264834,43461,272,272379,270188,42,Doritos Corn Chip Mexican Jalapeno 150g,2,7.8


In [12]:
pd.read_csv("../data/QVI_transaction_data.csv", encoding='latin-1', sep=",", on_bad_lines='skip')

ParserError: Error tokenizing data. C error: Buffer overflow caught - possible malformed input file.
